# Goal of the Notebook

The idea of this notebook is to perform experiments with respect to Feature Extraction.

Some ideas being :

- Adding otsu thresholding as an observation
- Adding disparity map between the two cameras as an observation
- Trying out different CNN architectures for feature extraction (+LSTM?)
- (Not exactly Feature Extraction, but trying out different algo types (DQN, etc))

### Imports

In [2]:
import sys
sys.path += ['../']

# image processing
import skimage
import cv2

# custom utils
from utils.image import get_otsu, get_disparity
from utils.imagewrapper import ImageWrap
from utils.models import linear_schedule, BasicExtractor

# math
import numpy as np 
import math

# plotting
import matplotlib.pyplot as plt
import utils

# project specific
import deepracer_gym
from agents.deepracer_base_agent import DeepracerAgent

# simulation and RL algo
import gym
from stable_baselines3 import A2C, PPO, DQN
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.vec_env import DummyVecEnv, VecTransposeImage, SubprocVecEnv, VecEnvWrapper, VecEnv
from stable_baselines3.common.vec_env.base_vec_env import VecEnv, VecEnvStepReturn, VecEnvWrapper
from stable_baselines3.common.vec_env import VecExtractDictObs
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import CheckpointCallback

# DL lib
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD

%matplotlib notebook

# PPO Model Test 

## Create Environment

In [7]:
# Make sure we use 32bits tensors
torch.set_default_dtype(torch.float32)

# Load Environment with wrapper func
start_env = lambda port : lambda: Monitor(ImageWrap(gym.make('deepracer_gym:deepracer-v0', port=port)))

# Start two envs
env = DummyVecEnv([start_env(8888), start_env(8889)])

# Save models every 50_000 steps
checkpoint_callback = CheckpointCallback(save_freq=50_000, save_path='./logs/',
                                         name_prefix='1M_model')

## Load Model

In [12]:
model = PPO.load('not_bad',
                 env=env,
                 learning_rate=linear_schedule(1e-4),
                 tensorboard_log="./a2c_new/",
                 device='cuda:0')

Wrapping the env in a VecTransposeImage.


## Train 

In [ ]:
model.learn(total_timesteps=600_000, callback=checkpoint_callback)